## Coffee Analysis for Sourcing

---

### Introduction
Coffee is one of the most important drinks humanity could have ever discovered and yet I feel it is also one of the more underappreciated things in regards to the broader population. After watching a bunch of tiktok and youtube short content creators attest to how much "better-tasting" coffee could be, I decided to take a leap of faith and try it at home for myself. And oh boy was this a rabbit hole that I never thought I would enjoy immensely or go down into.

At first, I was immediately able to taste the difference, flavors being quite smooth, non-bitter, and fruity after dialing in the right grind size. It was absolutely mind bending, "how could coffee taste this good!". However, I had a really hard time distinguishing the actual flavor profiles mentioned due to lack of tasting experience. Over time I was able to slowly differentiate some of the flavors (not all) and have been brewing coffee ever since my first buy (5 months to date). But there was always a thought in the back of my mind telling me to question "what more can you do to brew a better cup?" 

This question led me to confronting the most obvious factor in the entire process of making coffee "the origin" or for a better term "sourcing". Even though I am still a beginner in the space, it is a widely known that growing and processing methods at these coffee farms affect the outcome of the taste as well as the quality. However after some light digging, I was left with some unsatisfaction. Most of the articles put out there are very obscure or don't explain too much on how these factors come into play, or it felt like complete pseudoscience at times. 

To combat this I decided to take matters into my own hand and try and model something for myself to nail down what factors actually contribute to a good tasting cup of coffee. I told myself - "Welp, I am in a data science program after all, why not apply the skills I have learned so far and take a stab at it." Never have I ever once thought that I would be applying skills I have learned in my career path in this personal of a manner. Damn it! It's just a hobby bro.

Before we delve deeper into how I acquire and process the data here's some quick info on gear for those that are interested!

**At Home Setup**

- Chestnut C3 Grinder - TIMEMORE
- V60 Dripper - HARIO
- Origami Dripper Size M - ORIGAMI
- Origami Dripper Air Size S - ORIGAMI
- Respective V60 and Origami Filter Papers

---

### Methodology

Below we will follow standard steps in data science in defining and mapping out the process I will take.

**Objective/Question**
1. What factors contribute to a good tasting cup of coffee?
2. What specific regions can I nail down my coffee search to for coffee acquisition?

**Genreal Steps**
1. Acquiring Data
2. Exploratory Data Analysis (EDA)
3. Data Cleaning
4. Data Modeling + Evaluation
5. Visualization of Results

---

### Extra Crucial Information Before Starting

With the two stated questions above we need to dive into the details a little bit about what contributes to coffee scoring in this dataset so we don't become redundant or mess up our analysis.

It was noted in the dataset itself that total.cup.points being the scoring system is the sum of scorings on 10 attributes across 5 cups of the same coffee.
These Attributes are:

1. Aroma - scent or fragrance
2. Flavor - how well developed are the flavor notes present
3. Aftertaste - length of flavor once coffee has been swallowed, the longer the better scoring
4. Acidity - based on acidity scale, some coffees are expected to be more acidic, but in general the lower the better
5. Body - this is what is known as mouthfeel, 5 cups of the same coffee are graded on consistency of this feel
6. Balance - combination of (acidity, aroma, flavor, and aftertaste) the more they strike an equilibrium the higher the score
7. Sweetness - what is considered a balanced level of sweetness
8. Clean Cup - a derivation of uniformity from first sip to aftertaste
9. Uniformity - consistency of the uniformity of flavors among 5 of the same cups
10. Overall - Personal considerations and rating.

- Defects can set back the score 2-4 points during the cupping / tasting process.

**Final Scoring**

- 90 - 100 is outstanding
- 85 - 89.99 is excellent
- 80 - 84.99 is very good
- < 80  is below specialty coffee

Scoring Information: [Break Down of Coffee Scores](https://spiritanimalcoffee.com/blogs/spirit-animal-blog/the-coffee-quality-score)

**Notes on Scoring Process**

Since I have not tried many specialty coffees out yet, what I consider good may not be on par with how the SCAA (Specialty Coffee Association of America) thinks. Nor do I score my own at home cups in such a manner, because who really does that on a regular basis or at all?

Given this criteria I would like you the reader to keep in mind, the coffees I will be aiming for in our end result of *finding specific regions* will most likely have a threshold of what was shown 80+ to gauruntee specialty coffee standard, but if the results are quite limiting, I may expand the range to 65+ points.


---

### Imports Setup

In [53]:
# Basic libs
import pandas as pd
import numpy as np
from scipy.stats import *
import warnings
import json
from sklearn.preprocessing import LabelEncoder

# Plotting libs
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Warnings
warnings.filterwarnings("ignore")

---

### Data

Upon doing some extensive searching, it appears that a lot of coffee data that go into extreme detail are locked behind paywalls. Especially the ones from more official organizations like the Coffee Quality Institute (CQI). I have found the equivalent to what I was searching for on Kaggle, albeit the quality of the data may not be 100% reliable as it was scraped from CQI, but it seemed like the most reliable data I could get.

Here is the direct link to the data: [Coffee Quality Data (CQI)](https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi)

In [30]:
# reading in the dataset
cqi_df = pd.read_csv("../Data/cqi_2018/merged_data_cleaned.csv")
cqi_df.drop(columns=['Unnamed: 0'], inplace=True)

# Describing dataset
cqi_shape = "{} rows x {} columns".format(cqi_df.shape[0], cqi_df.shape[1])
print("Data Frame Shape: \n" + cqi_shape + " \n")
print("Columns: \n")
print(cqi_df.columns)

Data Frame Shape: 
1339 rows x 43 columns 

Columns: 

Index(['Species', 'Owner', 'Country.of.Origin', 'Farm.Name', 'Lot.Number',
       'Mill', 'ICO.Number', 'Company', 'Altitude', 'Region', 'Producer',
       'Number.of.Bags', 'Bag.Weight', 'In.Country.Partner', 'Harvest.Year',
       'Grading.Date', 'Owner.1', 'Variety', 'Processing.Method', 'Aroma',
       'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity',
       'Clean.Cup', 'Sweetness', 'Cupper.Points', 'Total.Cup.Points',
       'Moisture', 'Category.One.Defects', 'Quakers', 'Color',
       'Category.Two.Defects', 'Expiration', 'Certification.Body',
       'Certification.Address', 'Certification.Contact', 'unit_of_measurement',
       'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters'],
      dtype='object')


In [7]:
# Preview of data
cqi_df.head(2)

,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,Region,...,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0


---

### Exploratory Data Analysis

After we have the overview of the data let's start exploring the general distributions of our data. Below is a helper function that helps out with doing so with a handful of variables.

In [8]:
# Helper function for discovery of individual variables and their distributions
def quick_hist(df, col_name, asc, head, g_title=None):
    temp_counts = df[col_name].value_counts().reset_index(name='Counts').sort_values(['Counts'], ascending=asc)

    if head:
        temp_counts = temp_counts.head(head)
    
    #print("Tabling")
    #print(temp_counts)
    
    # Plotly
    if not g_title:
        g_title = f'{col_name} Distribution'

    f = px.bar(temp_counts, x=col_name, y='Counts', text='Counts', title=g_title, color=col_name)
    f.update_layout(width=800, height=600)

    f.show()

**Showcase Plotting for Distributions of Altitude, Harvest Year and Country of Origin**

Here are a few quick plots that showcase the usage of the above function. I use this mainly to check if there are any existing outliers or skewed distributions in the data. All in all it's interesting to see what our attributes are without having to scan a summary of inter quartile ranges.

Below features:
- Top 10 Altitude Distribution
- Top 10 Harvest Years
- Top 10 Country Origins

In [9]:
# Tester
#quick_hist(cqi_df, 'Species', False, False)
#quick_hist(cqi_df, 'Owner', False, 10, 'Top 10 Owner Distribution')

# Showcase
quick_hist(cqi_df, 'Altitude', False, 10, 'Top 10 Altitude Distribution')
quick_hist(cqi_df, 'Harvest.Year', False, 10, 'Top 10 Harvest Years')
quick_hist(cqi_df, 'Country.of.Origin', False, 10, 'Top 10 Country Origins')

**Plotting Altitude Highs vs Total Cup Points grouped by Country of Origin**

Let's proceed to explore a little further with different fields and how they influence Total.Cup.Points on a surface level.

The variables of interest are:
- Altitude
- Variety
- Processing.Method

Notes:
- Altitude vs Total Cup Points
  - A brief interactive scatter plot of how each entry scored in terms of total quality with the associated country labeled
  - It is hard to gain a direct insight from this but it is definitely interesting exploring it
- Variet vs Total Cup Points
  - It appears that each variety has a specific scoring range with some being wider than others.
  - Noteably out of the categorized ranges: Bourbon, Typica and Pacas seem to have the widest variation in scoring.
- Processing Method vs Total Cup Points
  - It seems like pulped natural and semi-washed processing methods have the most consistent scorings.
  - Washed / Wet method has the widest variation
  - All of the processing methods have a similar mean in score


In [10]:
# Quick outlier removal for better viewing
point_alt_view = cqi_df.copy()

z_score_alt = zscore(point_alt_view["altitude_high_meters"], nan_policy='omit')
z_score_p = zscore(point_alt_view["Total.Cup.Points"], nan_policy='omit')
thresh = 3

point_alt_view = point_alt_view[z_score_alt <= (thresh)]
point_alt_view = point_alt_view[abs(z_score_p) <= (thresh)]

# Plotting
fig = px.scatter(point_alt_view, x='altitude_mean_meters', y='Total.Cup.Points', title='Altitude High vs Total Cup Points by Country', color='Country.of.Origin')
fig.update_layout(width=800, height=600)
# Show the plot
fig.show()

In [11]:
# Variety on Total.Cup.Points
fig = px.box(point_alt_view, x='Variety', y='Total.Cup.Points', title='Variety vs Total Cup Points', color='Variety')
fig.update_layout(width=1200, height=600)
fig.show()

# Processing.Method on Total.Cup.Points
fig = px.box(point_alt_view, x='Processing.Method', y='Total.Cup.Points', title='Processing Method vs Total Cup Points', color='Processing.Method')
fig.update_layout(width=800, height=600)
fig.show()

---

### Checking General Distributions and Outliers

Let us check out the actual spread across all out variables before we do any data cleaning and analyze our variables in detail. This is necessary in determining if we need to cut variables or perhaps make new ones based on the existing ones.

In [27]:
# Check data types

# Numeric Predictors
num_cols = cqi_df.select_dtypes(include=['number']).columns
# Object predictors
obj_cols = cqi_df.select_dtypes(include=['object']).columns

# Printing Results
print("Numeric Columns:\n")
print(num_cols)
print("\nObject/String Columns:\n")
print(obj_cols)

Numeric Columns:

Index(['Number.of.Bags', 'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body',
       'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points',
       'Total.Cup.Points', 'Moisture', 'Category.One.Defects', 'Quakers',
       'Category.Two.Defects', 'altitude_low_meters', 'altitude_high_meters',
       'altitude_mean_meters'],
      dtype='object')

Object/String Columns:

Index(['Species', 'Owner', 'Country.of.Origin', 'Farm.Name', 'Lot.Number',
       'Mill', 'ICO.Number', 'Company', 'Altitude', 'Region', 'Producer',
       'Bag.Weight', 'In.Country.Partner', 'Harvest.Year', 'Grading.Date',
       'Owner.1', 'Variety', 'Processing.Method', 'Color', 'Expiration',
       'Certification.Body', 'Certification.Address', 'Certification.Contact',
       'unit_of_measurement'],
      dtype='object')


##### Numerical Data Spreads

In our numerical columns I believe most of these will have a resonable affect on modeling especially excluding the actual scoring contributions, however we will plot those factors just to see them. Of course we will be using Total.Cup.Points as an evaluation later on (predicted). The ones that have a wider range have been excluded and plotted separately, namely: ones that pertain to altitude. They are plotted in two parts as shown below.

A couple of side notes on exploration:
- All features that pertain to scoring will be removed in our final clean dataset, since they are already established as parts of what make up the final scoring.
- From a quick glance Moisture and Quakers do not show up well on this scale but will be used regardless.
- All three altitude variables are very similar and probably correlate heavily, will do some further evaluation and end up picking the mean value.

In [13]:
# Selecting all numeric variables
num_df = cqi_df.select_dtypes(include=['number'])

# Excluding variables that have too high or low of a range causing plot to look bad
num_df = num_df.loc[:, ~num_df.columns.isin(['altitude_low_meters', 'altitude_mean_meters', 
                                             'altitude_high_meters', 'Number.of.Bags', 'Total.Cup.Points',
                                               'Category.One.Defects', 'Category.Two.Defects'])]

# Plotting
fig = make_subplots(rows=(len(num_df.columns) + 2) // 4, cols=4, subplot_titles=num_df.columns)

# Indexing for subplots
n_col = 1
x_col = 1

# Loop through all selected columns
for i, column in enumerate(num_df.columns, start=1):
    
    # adding trace to subplots
    fig.add_trace(
        go.Histogram(x=num_df[column], name=column, nbinsx=15),
        row=x_col, col=n_col
    )

    # update logic for subplot indexing
    if i % 3 == 0:
        n_col = int(i / 3) + 1
        x_col = 1
    else:
        x_col += 1

# Update layout
fig.update_layout(height=800, width=1000, title_text="All Continuous Variables")
fig.show()

In [14]:
# Altitude Numericals Boxplot using point_alt_view which has taken out the outliers
f = px.box(point_alt_view[['altitude_low_meters', 'altitude_mean_meters', 'altitude_high_meters']], title='Box Plot for Altitude Attributes')
f.update_layout(width=800, height=600)
f.show()

##### Categorical Data Spreads

I won't spam graphs down the page since there are a lot of these variables but I will provide two more interesting plots to showcase and provide a summary of my findings from looking at all the distributions.
species distribution.

Through looking at the data I have hand picked some variables that make sense and excluded others that do not matter. For example: In.Country.Partner and Certification.Body , are both variables that have to do with external information so I will only pick one to see if there is an influence in modeling later to see if there is a bias in grading, I will go with the later. Others such as altitude have other variable counter parts, in this particular case 3 defining (low, high, mean) in the above section. Dated variables such as harvest year will be used but others that provide a pure date may be a little too continuous to categorize. To account for a possible seasonal element I may go with seperating grading date into year and months as well as performing a calculation on grading date - Expiration to provide a more useful metric days to expiration. All the other variables seem reasonable and can be categorized easily through Label Encoding, I will probably be using the sklearn.preprocessing LabelEncoder feature.

We will keep all the features below and let the further modeling processes filter out more:

- country of origin
- bag weight (bucket)
- harvest year
- grading date (seperate to grading year)
- variety
- processing method
- color
- grading date - Expiration (dates to expiry)
- Certification.Body


The following below will plot out interesting factors that the coffee community notes frequently (or rather puts a heavier weight on)

In [15]:
# Exploration on the categorical data spreads

# Looking through all variables in brute force quick_hist method
# categorical_select = ['Species', 'Country.of.Origin', 'Altitude', 'Region',
#        'Bag.Weight', 'In.Country.Partner', 'Harvest.Year', 'Grading.Date',
#         'Variety', 'Processing.Method', 'Color', 'Expiration',
#        'Certification.Body']

# for i in categorical_select:
#     quick_hist(cqi_df, i, False, False)

# Most interesting and notable attributes mentioned by the coffee community
categorical_select = ['Processing.Method', 'Variety', 'Color']

# Loop through and plot each in a stacked bar chart
for i in categorical_select:

    processing_df = cqi_df[[i, 'Country.of.Origin']]

    p_view = processing_df.groupby(['Country.of.Origin', i]).size().reset_index(name='Counts').sort_values(by=['Counts'],ascending=[False])

    f = px.bar(p_view, x='Country.of.Origin', y='Counts', color=i, title=f'Countries vs {i}')
    f.show()


---

### Data Cleaning

Okay cool! We know exactly what we want now through the analysis above. So below is how we shall approach data cleaning.

The process will be as follows:
- Outlier removal for all numerical data (again we are including everything except the exact scoring metrics)
- Using categorical features of [Country of originm, bag weight (bucket), harvest year, grading date (seperate to grading year), variety, processing method, color, dates_to_expiry (calculation), Certification.Body]
- Outlier removal of categorical features and additional calculated fields, Label Encoding.

In [108]:
# Select wanted columns only
filtered_preds = ['Species', 'Country.of.Origin', 'Region',
       'Bag.Weight', 'In.Country.Partner', 'Harvest.Year', 'Grading.Date',
       'Variety', 'Processing.Method', 'Color', 'Expiration',
       'Certification.Body', 'Number.of.Bags', 'Total.Cup.Points', 'Moisture', 'Category.One.Defects', 'Quakers',
       'Category.Two.Defects', 'altitude_mean_meters']

# Make a copy of the old dataframe with only wanted values
cqi_clean = cqi_df.copy()
cqi_clean = cqi_clean[filtered_preds]

print("Shape: " + str(cqi_clean.shape))

Shape: (1339, 19)


In [109]:
# Checks for na values ---
def check_na_cols(temp_df):
    na_cols = temp_df.columns[temp_df.isna().any()]
    na_print = []

    if len(na_cols) > 0:
        for var in na_cols:
            na_print.append(var)

    print(f"NA Detected in [{len(na_print)} / {len(temp_df.columns)}] Columns: ")
    print(na_print)

    print("Respective Counts: ")
    na_counts = temp_df.isna().sum()
    na_counts = na_counts[na_counts != 0]
    print(na_counts)

    print("Shape Check: " + str(temp_df.shape) + "\n")

print("--- Initial Check ---")
check_na_cols(cqi_clean)


# Address NA values ---

# Country.of.Origin - removal of entries
cqi_clean = cqi_clean.dropna(subset=['Country.of.Origin'])

# Region - categorize to other
cqi_clean['Region'] = cqi_clean['Region'].fillna('other')

# Harvest Year - removal of entries
cqi_clean = cqi_clean.dropna(subset=['Harvest.Year'])

# Variety - categorize to other
cqi_clean['Variety'] = cqi_clean['Variety'].fillna('other')

# Processing.Method - categorize to other
cqi_clean['Processing.Method'] = cqi_clean['Processing.Method'].fillna('other')

# Color - categorize to other
cqi_clean['Color'] = cqi_clean['Color'].fillna('other')

# Quakers - remove
cqi_clean = cqi_clean.dropna(subset=['Quakers'])

# Quick Check
print("--- Secondary Check ---")
check_na_cols(cqi_clean)

# altitude_mean_meters
# Due to the nature of the data it appears that going with [altitude mean meters] is better, there are a lof rows that have a singular value on range.
# Altitude by itself is very inconsistent in terms of units and state the same things as a range, harder to parse consistently.
#cqi_clean[['altitude_mean_meters', 'altitude_high_meters', 'altitude_low_meters', 'Altitude']].tail(10)
cqi_clean = cqi_clean.dropna(subset=['altitude_mean_meters'])

# Grading Date - Expiration and removal of these two variables
def date_parse(d):
    formats = ['st,', 'nd,', 'rd,', 'th,']
    for i in formats:
        d = d.replace(i, '')

    d = d.strip()
    try:
        # Parse the date string into a datetime object
        return pd.to_datetime(d, format='%B %d %Y')
    except ValueError:
        # Print the date string that failed to parse
        print(f"Failed to parse date string: '{d}'")
        return None

#cqi_clean['Expiration'] = cqi_clean['Expiration'].apply(date_parse)
cqi_clean['Grading.Date'] = cqi_clean['Grading.Date'].apply(date_parse)

#cqi_clean['days_to_expiry'] = cqi_clean["Expiration"] - cqi_clean['Grading.Date']
#print(cqi_clean['days_to_expiry'])
#print(len(cqi_clean['days_to_expiry'].unique()))

# After performing a check on this metric it seems like all days to expiry is 365 so I will be seperating into two columns instead
# 1. Grading Year
# 2. Grading Month

cqi_clean['grading_year'] = cqi_clean['Grading.Date'].dt.year
cqi_clean['grading_month'] = cqi_clean['Grading.Date'].dt.month
cqi_clean = cqi_clean.drop(columns=['Grading.Date', 'Expiration'])

# Final Check
print("--- Third Check ---")
check_na_cols(cqi_clean)

--- Initial Check ---
NA Detected in [8 / 19] Columns: 
['Country.of.Origin', 'Region', 'Harvest.Year', 'Variety', 'Processing.Method', 'Color', 'Quakers', 'altitude_mean_meters']
Respective Counts: 
Country.of.Origin         1
Region                   59
Harvest.Year             47
Variety                 226
Processing.Method       170
Color                   270
Quakers                   1
altitude_mean_meters    230
dtype: int64
Shape Check: (1339, 19)

--- Secondary Check ---
NA Detected in [1 / 19] Columns: 
['altitude_mean_meters']
Respective Counts: 
altitude_mean_meters    195
dtype: int64
Shape Check: (1291, 19)

--- Third Check ---
NA Detected in [0 / 19] Columns: 
[]
Respective Counts: 
Series([], dtype: int64)
Shape Check: (1096, 19)



In [110]:
# Encoding categoricals
def mass_encode(temp_df):
    
    cat_cols = temp_df.select_dtypes(include=['object']).columns
    print("Calling Label Encoding On: ")
    print(cat_cols)

    labs_encoding = {}
    encoded_map = {}

    for i in cat_cols:
        
        labs_encoding[i] = LabelEncoder()
        temp_df[i] = labs_encoding[i].fit_transform(temp_df[i])        
        encoded_map[i] = dict(zip(labs_encoding[i].classes_, labs_encoding[i].transform(labs_encoding[i].classes_)))


    return temp_df, encoded_map

cqi_clean, cqi_encoding_map = mass_encode(cqi_clean)

print("Encoding Dictionary: ")
print(cqi_encoding_map)
print("Sample of Encoding in cqi_clean: ")
cqi_clean.head(5)

Calling Label Encoding On: 
Index(['Species', 'Country.of.Origin', 'Region', 'Bag.Weight',
       'In.Country.Partner', 'Harvest.Year', 'Variety', 'Processing.Method',
       'Color', 'Certification.Body'],
      dtype='object')
Encoding Dictionary: 
{'Species': {'Arabica': 0, 'Robusta': 1}, 'Country.of.Origin': {'Brazil': 0, 'Burundi': 1, 'China': 2, 'Colombia': 3, 'Costa Rica': 4, 'Cote d?Ivoire': 5, 'Ecuador': 6, 'El Salvador': 7, 'Ethiopia': 8, 'Guatemala': 9, 'Haiti': 10, 'Honduras': 11, 'India': 12, 'Indonesia': 13, 'Kenya': 14, 'Laos': 15, 'Malawi': 16, 'Mauritius': 17, 'Mexico': 18, 'Myanmar': 19, 'Nicaragua': 20, 'Panama': 21, 'Papua New Guinea': 22, 'Peru': 23, 'Philippines': 24, 'Rwanda': 25, 'Taiwan': 26, 'Tanzania, United Republic Of': 27, 'Thailand': 28, 'Uganda': 29, 'United States': 30, 'United States (Hawaii)': 31, 'United States (Puerto Rico)': 32, 'Vietnam': 33, 'Zambia': 34}, 'Region': {'52 narino (exact location: mattituy; municipal region: florida code 381': 0, 'a

,Species,Country.of.Origin,Region,Bag.Weight,In.Country.Partner,Harvest.Year,Variety,Processing.Method,Color,Certification.Body,Number.of.Bags,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Category.Two.Defects,altitude_mean_meters,grading_year,grading_month
0,0,8,106,37,14,13,28,4,2,14,300,90.58,0.12,0,0.0,0,2075.0,2015,4
1,0,8,106,37,14,13,14,4,2,14,300,89.92,0.12,0,0.0,1,2075.0,2015,4
3,0,8,211,37,14,13,28,0,2,14,320,89.00,0.11,0,0.0,2,2000.0,2015,3
4,0,8,106,37,14,13,14,4,2,14,300,88.83,0.12,0,0.0,2,2075.0,2015,4
7,0,8,211,37,11,37,28,5,3,11,300,88.67,0.03,0,0.0,0,1635.0,2010,9


In [111]:
# Correlation Plot
cqi_clean_corr = cqi_clean.corr()

fcorr = px.imshow(cqi_clean_corr,
                labels=dict(x="Columns", y="Columns", color="Correlation"),
                x=cqi_clean_corr.columns,
                y=cqi_clean_corr.columns)

fcorr.update_layout(title='Correlation of cqi_clean')
fcorr.update_layout(width=1500, height=1000)
fcorr.show()

**Notes:**

- It appears that Certification.Body has a high correlation with In.Country.Partner, we will opt to In.Country.Partner
- Everything else looks relatively good so we will let other feature selection methods take over in filtering out the rest

In [ ]:
# Taking out Certification.Body

In [19]:
# Addressing outliers

In [22]:
# Finalizing and saving new version of dataset

---

### Data Modeling

In [23]:
# Basis  ---
# Linear Regression

# Does it make sense to do a log transformation

# Variable selection methods ---

# Stewpise Backwards + Forwards

# LASSO + RIDGE + ELASTIC

# Other models ---

# Random Forest

# Tuned Random Forest

# KNN

# Add one final model

# ALSO ADDRESS FEATURE IMPORTANCE IN THE MODELS.

In [24]:
# Table the model results for comparison

---

### Visualizations

In [25]:
# Using best model for predictions based on scoring and their associated altitudes. - Reverse predict

# What is the ideal things to search on


# Mapping those results to a geographical data to create a heat map of the best scoring regions

---

### Thoughts & Conclusions

---